In [2]:
import numpy as np
import pandas as pd
import importlib


import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import manhattan_distances

import tcrvalid.load_models as mdl
importlib.reload(mdl)
from tcrvalid.physio_embedding import SeqArrayDictConverter
from tcrvalid.data_subsetting import *
from tcrvalid.cluster_loop import *
from tcrvalid.defaults import *
from tcrvalid.physio_embedding import *
from tcrvalid.plot_utils import *

Populated Model Dictionaries
{'0_0': 'C:\\Users\\admin\\Documents\\GitHub\\tcrvalid\\tcrvalid\\logged_models\\TRB\\0_0\\decoder_kr', '10_1': 'C:\\Users\\admin\\Documents\\GitHub\\tcrvalid\\tcrvalid\\logged_models\\TRB\\10_1\\decoder_kr', '1_1': 'C:\\Users\\admin\\Documents\\GitHub\\tcrvalid\\tcrvalid\\logged_models\\TRB\\1_1\\decoder_kr', '1_10': 'C:\\Users\\admin\\Documents\\GitHub\\tcrvalid\\tcrvalid\\logged_models\\TRB\\1_10\\decoder_kr', '1_2': 'C:\\Users\\admin\\Documents\\GitHub\\tcrvalid\\tcrvalid\\logged_models\\TRB\\1_2\\decoder_kr', '1_2_full_40': 'C:\\Users\\admin\\Documents\\GitHub\\tcrvalid\\tcrvalid\\logged_models\\TRB\\1_2_full_40\\decoder_kr', '1_5': 'C:\\Users\\admin\\Documents\\GitHub\\tcrvalid\\tcrvalid\\logged_models\\TRB\\1_5\\decoder_kr'}
Populated Model Dictionaries
{'0_0': 'C:\\Users\\admin\\Documents\\GitHub\\tcrvalid\\tcrvalid\\logged_models\\TRB\\0_0\\decoder_kr', '10_1': 'C:\\Users\\admin\\Documents\\GitHub\\tcrvalid\\tcrvalid\\logged_models\\TRB\\10_1\\deco

In [3]:
sample_df=pd.read_csv("../../tcrdist_graph_data/YLQ_filter_vgene.csv")
sample_df

,epitopes_aa,epitopes_hla,cdr3_a_aa,cdr3_b_aa,epitope_species,epitope_gene,source,validated,v_a_gene,v_b_gene
0,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNRGDKLVF,CATGDSNTGELFF,Covid19,S,Xie_2025,True,TRAV12-1,TRBV7-8
1,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNRDDKIIF,CARSDANTGELFF,Covid19,S,Xie_2025,True,TRAV12-1,TRBV7-8
2,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNMPQAGTALIF,CAILADMNTGELFF,Covid19,S,Xie_2025,True,TRAV12-1,TRBV10-3
3,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNNEDDMRF,CATEAVNTGELFF,Covid19,S,Xie_2025,True,TRAV12-1,TRBV24-1
4,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNNNDKLIF,CASSDGNTGELFF,Covid19,S,Xie_2025,True,TRAV12-1,TRBV7-9
...,...,...,...,...,...,...,...,...,...,...
589,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNGNNDMRF,CARQDSNTGELFF,Covid19,S,TCRvdb,True,TRAV12-1,TRBV12-3
590,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNMPGAGSYQLTF,CASSPVAGGNTGELFF,Covid19,S,TCRvdb,True,TRAV12-1,TRBV13
591,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNSDTDKLIF,CASGGTNTGELFF,Covid19,S,TCRvdb,True,TRAV12-1,TRBV2
592,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNNNNDMRF,CARGDANTGELFF,Covid19,S,TCRvdb,True,TRAV12-1,TRBV12-3


In [10]:
trb_ref_df=pd.read_csv("../../tcrvalid/data/TRBV_reference.csv").drop(columns="Unnamed: 0").rename(
    columns={"new_v_call":"v_b_new_call",
             "new_meta_vcall":"v_b_gene",
             "cdr1_no_gaps":"b_cdr1",
             "cdr2_no_gaps":"b_cdr2",
             "cdr1_cdr2_no_gaps":"b_cdr1-cdr2"
             })
trb_ref_df

,v_b_new_call,v_b_gene,b_cdr1,b_cdr2,b_cdr1-cdr2
0,TRBV1*01,TRBV1,LGHDS,YNCKEF,LGHDS-YNCKEF
1,TRBV10-1*01,TRBV10-1,WNHNN,SYGVQD,WNHNN-SYGVQD
2,TRBV10-2*01,TRBV10-2,WSHSY,SAAADI,WSHSY-SAAADI
3,TRBV10-3*01,TRBV10-3,ENHRY,SYGVKD,ENHRY-SYGVKD
4,TRBV11-1*01,TRBV11-1,SGHAT,FQDESV,SGHAT-FQDESV
...,...,...,...,...,...
61,TRBV7-6*01,TRBV7-6,SGHVS,FNYEAQ,SGHVS-FNYEAQ
62,TRBV7-7*01,TRBV7-7,SSHAT,FNYEAQ,SSHAT-FNYEAQ
63,TRBV7-8*01,TRBV7-8,SGHVS,FQNEAQ,SGHVS-FQNEAQ
64,TRBV7-9*01,TRBV7-9,SEHNR,FQNEAQ,SEHNR-FQNEAQ


In [11]:
sample_df=sample_df.merge(trb_ref_df, how='left',on="v_b_gene")
sample_df

,epitopes_aa,epitopes_hla,cdr3_a_aa,cdr3_b_aa,epitope_species,epitope_gene,source,validated,v_a_gene,v_b_gene,v_b_new_call,b_cdr1,b_cdr2,b_cdr1-cdr2
0,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNRGDKLVF,CATGDSNTGELFF,Covid19,S,Xie_2025,True,TRAV12-1,TRBV7-8,TRBV7-8*01,SGHVS,FQNEAQ,SGHVS-FQNEAQ
1,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNRDDKIIF,CARSDANTGELFF,Covid19,S,Xie_2025,True,TRAV12-1,TRBV7-8,TRBV7-8*01,SGHVS,FQNEAQ,SGHVS-FQNEAQ
2,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNMPQAGTALIF,CAILADMNTGELFF,Covid19,S,Xie_2025,True,TRAV12-1,TRBV10-3,TRBV10-3*01,ENHRY,SYGVKD,ENHRY-SYGVKD
3,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNNEDDMRF,CATEAVNTGELFF,Covid19,S,Xie_2025,True,TRAV12-1,TRBV24-1,TRBV24-1*01,KGHDR,SFDVKD,KGHDR-SFDVKD
4,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNNNDKLIF,CASSDGNTGELFF,Covid19,S,Xie_2025,True,TRAV12-1,TRBV7-9,TRBV7-9*01,SEHNR,FQNEAQ,SEHNR-FQNEAQ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNGNNDMRF,CARQDSNTGELFF,Covid19,S,TCRvdb,True,TRAV12-1,TRBV12-3,TRBV12-3*01,SGHNS,FNNNVP,SGHNS-FNNNVP
590,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNMPGAGSYQLTF,CASSPVAGGNTGELFF,Covid19,S,TCRvdb,True,TRAV12-1,TRBV13,TRBV13*01,PRHDT,FYEKMQ,PRHDT-FYEKMQ
591,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNSDTDKLIF,CASGGTNTGELFF,Covid19,S,TCRvdb,True,TRAV12-1,TRBV2,TRBV2*01,SNHLY,FYNNEI,SNHLY-FYNNEI
592,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNNNNDMRF,CARGDANTGELFF,Covid19,S,TCRvdb,True,TRAV12-1,TRBV12-3,TRBV12-3*01,SGHNS,FNNNVP,SGHNS-FNNNVP


In [15]:
sample_df["b_cdr2-cdr3"]=sample_df.apply(lambda row: f"{row['b_cdr2']}-{row['cdr3_b_aa']}",axis=1)
sample_df

,epitopes_aa,epitopes_hla,cdr3_a_aa,cdr3_b_aa,epitope_species,epitope_gene,source,validated,v_a_gene,v_b_gene,v_b_new_call,b_cdr1,b_cdr2,b_cdr1-cdr2,b_cdr2-cdr3
0,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNRGDKLVF,CATGDSNTGELFF,Covid19,S,Xie_2025,True,TRAV12-1,TRBV7-8,TRBV7-8*01,SGHVS,FQNEAQ,SGHVS-FQNEAQ,FQNEAQ-CATGDSNTGELFF
1,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNRDDKIIF,CARSDANTGELFF,Covid19,S,Xie_2025,True,TRAV12-1,TRBV7-8,TRBV7-8*01,SGHVS,FQNEAQ,SGHVS-FQNEAQ,FQNEAQ-CARSDANTGELFF
2,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNMPQAGTALIF,CAILADMNTGELFF,Covid19,S,Xie_2025,True,TRAV12-1,TRBV10-3,TRBV10-3*01,ENHRY,SYGVKD,ENHRY-SYGVKD,SYGVKD-CAILADMNTGELFF
3,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNNEDDMRF,CATEAVNTGELFF,Covid19,S,Xie_2025,True,TRAV12-1,TRBV24-1,TRBV24-1*01,KGHDR,SFDVKD,KGHDR-SFDVKD,SFDVKD-CATEAVNTGELFF
4,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNNNDKLIF,CASSDGNTGELFF,Covid19,S,Xie_2025,True,TRAV12-1,TRBV7-9,TRBV7-9*01,SEHNR,FQNEAQ,SEHNR-FQNEAQ,FQNEAQ-CASSDGNTGELFF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNGNNDMRF,CARQDSNTGELFF,Covid19,S,TCRvdb,True,TRAV12-1,TRBV12-3,TRBV12-3*01,SGHNS,FNNNVP,SGHNS-FNNNVP,FNNNVP-CARQDSNTGELFF
590,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNMPGAGSYQLTF,CASSPVAGGNTGELFF,Covid19,S,TCRvdb,True,TRAV12-1,TRBV13,TRBV13*01,PRHDT,FYEKMQ,PRHDT-FYEKMQ,FYEKMQ-CASSPVAGGNTGELFF
591,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNSDTDKLIF,CASGGTNTGELFF,Covid19,S,TCRvdb,True,TRAV12-1,TRBV2,TRBV2*01,SNHLY,FYNNEI,SNHLY-FYNNEI,FYNNEI-CASGGTNTGELFF
592,"GYLQPRTFLL,VGYLQPRTF,YLQPRTFLL,YLQPRTFLLK,YLRP...","A*02:01,A*24:02,B*07:02",CVVNNNNDMRF,CARGDANTGELFF,Covid19,S,TCRvdb,True,TRAV12-1,TRBV12-3,TRBV12-3*01,SGHNS,FNNNVP,SGHNS-FNNNVP,FNNNVP-CARGDANTGELFF


In [17]:
list(sample_df["b_cdr2-cdr3"])

['FQNEAQ-CATGDSNTGELFF',
 'FQNEAQ-CARSDANTGELFF',
 'SYGVKD-CAILADMNTGELFF',
 'SFDVKD-CATEAVNTGELFF',
 'FQNEAQ-CASSDGNTGELFF',
 'SVGEGT-CANQDLNTGELFF',
 'FQNEAQ-CASSFQNTGELFF',
 'SNTAGT-CASQELNTGELFF',
 'FYNNEI-CARQFMNTGELFF',
 'FYNNEI-CAIQDGNTGELFF',
 'FQNEAQ-CASNNPNTGELFF',
 'FQNEAQ-CASSDSYGYTF',
 'FNNNVP-CAQGQGNTGELFF',
 'FQNEAQ-CARGRPNTGELFF',
 'SQIVND-CASQDSNTGELFF',
 'SQIVND-CASQTVNTGELFF',
 'FQNEAQ-CASSSDIEAFF',
 'SQIVND-CASSILASTDTQYF',
 'LQKENI-CASSILASTDTQYF',
 'FQNEAQ-CASSPDIEAFF',
 'SFDVKD-CAAEDRTNTGELFF',
 'SFDVKD-CATGNANTGELFF',
 'FQNEAQ-CASSPDIEAFF',
 'SVGIG-CAYLDSNTGELFF',
 'FNNNVP-CAFPSNYNTGELFF',
 'FQNEAQ-CASSDSMGYTF',
 'FQNEAQ-CASSLDIVQFF',
 'SFDVKD-CASRGPNTGELFF',
 'SQIVND-CASSPLGGGNTGELFF',
 'SQIVND-CASSILASTDTQYF',
 'FQNEAQ-CASSLHYGEQYF',
 'FQNEAQ-CASSDSMGYTF',
 'FQNEAQ-CASSDSLGYTF',
 'SQIVND-CASLPDLNTGELFF',
 'FQNEAQ-CASNEGNTGELFF',
 'FQNEAQ-CASSPEIEAFF',
 'FQNEAQ-CASSPDIEAFF',
 'FQNEAQ-CAPGDANTGELFF',
 'SVGIG-CAWSVLPLGGGELFF',
 'FQNEAQ-CASSPDIEAFF',
 'SNEGSKA-CSA

In [24]:
longest=max(list(sample_df["b_cdr2-cdr3"]),key=len)
len(longest)

25

In [18]:
from_keras = True #expect True in github code

model_names = {
    'TRB': ['1_2_full_40'],
    
}

In [19]:
mapping = SeqArrayDictConverter()

loaded_trb_models = mdl.load_named_models(model_names['TRB'],chain='TRB',as_keras=from_keras)
loaded_decoders = mdl.load_named_models(model_names['TRB'],chain='TRB',as_keras=from_keras,encoders=False)

encoder = loaded_trb_models['1_2_full_40']
decoder = loaded_decoders['1_2_full_40']

{'0_0': 'C:\\Users\\admin\\Documents\\GitHub\\tcrvalid\\tcrvalid\\logged_models\\TRB\\0_0\\encoder_kr', '10_1': 'C:\\Users\\admin\\Documents\\GitHub\\tcrvalid\\tcrvalid\\logged_models\\TRB\\10_1\\encoder_kr', '1_1': 'C:\\Users\\admin\\Documents\\GitHub\\tcrvalid\\tcrvalid\\logged_models\\TRB\\1_1\\encoder_kr', '1_10': 'C:\\Users\\admin\\Documents\\GitHub\\tcrvalid\\tcrvalid\\logged_models\\TRB\\1_10\\encoder_kr', '1_2': 'C:\\Users\\admin\\Documents\\GitHub\\tcrvalid\\tcrvalid\\logged_models\\TRB\\1_2\\encoder_kr', '1_2_full_40': 'C:\\Users\\admin\\Documents\\GitHub\\tcrvalid\\tcrvalid\\logged_models\\TRB\\1_2_full_40\\encoder_kr', '1_5': 'C:\\Users\\admin\\Documents\\GitHub\\tcrvalid\\tcrvalid\\logged_models\\TRB\\1_5\\encoder_kr'}
{'0_0': 'C:\\Users\\admin\\Documents\\GitHub\\tcrvalid\\tcrvalid\\logged_models\\TRB\\0_0\\encoder_kr', '10_1': 'C:\\Users\\admin\\Documents\\GitHub\\tcrvalid\\tcrvalid\\logged_models\\TRB\\10_1\\encoder_kr', '1_1': 'C:\\Users\\admin\\Documents\\GitHub\\tcrv

In [25]:
mapping = SeqArrayDictConverter()

# convert seq to physicochemical - list could have many sequences
# sequences are "CDR2-CDR3" formatted
x = mapping.seqs_to_array(list(sample_df["b_cdr2-cdr3"])
,maxlen=28)

# get TCR-VALID representation
z,_,_ = encoder.predict(x)
print(z)

[[-0.23917677  0.5080883   0.19604501 ...  1.8715179   0.48766392
  -0.169848  ]
 [-0.24086143  0.47023118 -0.7044405  ...  0.9701098   0.48408943
  -0.01368782]
 [-0.94597137  0.34762588  2.4943209  ... -0.7316117   0.17500989
  -0.07266119]
 ...
 [-0.24414276 -1.5000936  -0.7294164  ...  2.0378766   0.49188948
  -0.12650672]
 [-0.24226035 -0.44138023 -1.4520602  ...  1.9018617   0.48806262
  -0.16689429]
 [ 1.2036017  -1.5960499   0.27946973 ...  0.19506702  0.9070264
  -0.48416272]]


In [22]:
z.shape

(594, 16)

In [26]:
to_save=np.array(z)
to_save

array([[-0.23917677,  0.5080883 ,  0.19604501, ...,  1.8715179 ,
         0.48766392, -0.169848  ],
       [-0.24086143,  0.47023118, -0.7044405 , ...,  0.9701098 ,
         0.48408943, -0.01368782],
       [-0.94597137,  0.34762588,  2.4943209 , ..., -0.7316117 ,
         0.17500989, -0.07266119],
       ...,
       [-0.24414276, -1.5000936 , -0.7294164 , ...,  2.0378766 ,
         0.49188948, -0.12650672],
       [-0.24226035, -0.44138023, -1.4520602 , ...,  1.9018617 ,
         0.48806262, -0.16689429],
       [ 1.2036017 , -1.5960499 ,  0.27946973, ...,  0.19506702,
         0.9070264 , -0.48416272]], dtype=float32)

In [29]:
np.savetxt("../../tcrdist_graph_data/embedding_outputs/YLQ_test_embeddings",to_save,delimiter=" ")

In [30]:
to_load=np.loadtxt("../../tcrdist_graph_data/embedding_outputs/YLQ_test_embeddings",delimiter=" ")
to_load

array([[-0.23917677,  0.50808829,  0.19604501, ...,  1.8715179 ,
         0.48766392, -0.169848  ],
       [-0.24086143,  0.47023118, -0.70444047, ...,  0.97010982,
         0.48408943, -0.01368782],
       [-0.94597137,  0.34762588,  2.49432087, ..., -0.73161173,
         0.17500989, -0.07266119],
       ...,
       [-0.24414276, -1.50009358, -0.72941637, ...,  2.03787661,
         0.49188948, -0.12650672],
       [-0.24226035, -0.44138023, -1.45206022, ...,  1.90186167,
         0.48806262, -0.16689429],
       [ 1.20360172, -1.5960499 ,  0.27946973, ...,  0.19506702,
         0.90702641, -0.48416272]])